# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


---

In [ ]:
import requests

In [ ]:
import pandas as pd

In [ ]:
import os

---

# Внимание
При запуске ноутбука - будьте осторожны, уникальных объектов более 10.000, и внутри каждого объекта список фотографий.

В папке проекта будет создано столько же папок со всеми фотографиями внутри.

Для тестирования переменную limit_ советуется установить = 1 

---

# Скачиваем список строящихся объектов
Фильтрация строящихся объектов происходит в API запросе по фильтру objStatus=0

offset - начальная позиция с которой забираем данные

limit  - количество данных за запрос

In [ ]:
i = 0 # небольшая переменная для подстраховки при перечислении
offset_ = 0 # с какой точки берем данные в каждом запросе
limit_ = 1 # как много брать данных за один запрос
objects_list = [] # полученные данные

while i < 20:
    # Формируем URL для скачивания данных
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?' \
          f'offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    # Складываем полученные данные из запроса в общий список
    objects_list_temp = objects_data.get('data').get('list')
    objects_list.extend(objects_list_temp)
    # Если данных стало меньше чем лимит - значит данные подошли к концу
    offset_ += limit_ + 1
    i += 1
    if len(objects_list_temp) < limit_:
        break

## Создаем DataFrame со всеми данными строящихся домов

In [ ]:
df = pd.json_normalize(objects_list)

---

## Сохраняем полученные данные в Excel, Pickle, DB SQL

In [ ]:
df.to_excel("НашДомРФ.xlsx",sheet_name="construction")
#df_excel = pd.read_excel("НашДомРФ.xlsx")
#df_excel

In [ ]:
df.to_pickle("НашДомРФ.pkl")
#df_pickle = pd.read_pickle("НашДомРФ.pkl")
#df_pickle

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

df.to_sql('construction',engine)
#engine.execute("SELECT * FROM construction").fetchall()

---

# Скачиваем фотографии каждого объекта в отдельную папку

In [ ]:
# Получаем уникальные идентификаторы для формирования каждого отдельного запроса
objids = [x.get('objId') for x in objects_list]
for objId in objids:
    # Формируем URL для скачивания данных
    building_url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B' \
                   f'/api/object/{objId}'
    building_res = requests.get(building_url)
    building_data = building_res.json()
    # Формируем уникальное имя для папки (ID и адрес)
    building_address = str(building_data.get('data').get('id')) + ': ' + building_data.get('data').get('address')
    # Проверяем существует ли уже такая папка с данными, если нет - создаем
    isExist = os.path.exists(building_address)
    if not isExist:
        os.makedirs(building_address, exist_ok=False)
    # Получаем список данных фотографий объекта
    building_photos = building_data.get('data').get('photoRenderDTO')
    for building_photo in building_photos:
        # Формируем URL для скачивания данных
        building_photo_url = building_photo.get('objRenderPhotoUrl')
        building_photo_file_name = building_address + '/' + building_photo.get('objRenderPhotoNm')
        res = requests.get(building_photo_url)
        # Сохраняем каждую фотографию в отдельную папку объекта
        with open(f'{building_photo_file_name}', 'wb') as f:
            f.write(res.content)